# Obtain Genome Window (40~100 genes per window)

Import X matrix (del or  dup)

In [1]:
import pandas as pd, numpy as np
import os
from pandasql import sqldf
from collections import Counter
from more_itertools import unique_everseen
from datetime import datetime
cwd = os.path.expanduser("~/GIT/cnv-gene-mapping/data")

In [2]:
n_block = 449
n_gene_per_block = 20

In [6]:
dele = pd.read_table(f"{cwd}/deletion.gz", compression = "gzip", sep = "\t", header = 0)

In [4]:
dele.head()

,Unnamed: 0,PT-L1HP,PT-8TK1,PT-8UY5,PT-BQF5,PT-8WFK,PT-BQCA,PT-8W5I,PT-2M8I,PT-BRH4,PT-BQOG,PT-BSBC,PT-FFMB,PT-ONJC,PT-ESML,PT-ITQB,PT-IS2P,PT-ISF4,PT-L17S,PT-OO5U,PT-8W9I,PT-BRVD,PT-9ZJH,PT-8VH6,PT-ESFV,PT-FH16,PT-L1BS,PT-ISIV,PT-ITQT,PT-IT5X,PT-L1LJ,PT-L1HU,PT-L1RU,PT-L18K,PT-P2AM,PT-1SBR,PT-9ZIP,PT-8VT3,PT-COOS,PT-OOLJ,...,PT-8WJS,PT-1S18,PT-1S7Q,PT-1SBT,PT-1TBE,PT-1TC6,PT-1S8F,PT-1S82,PT-1S2M,PT-1TCU,PT-1RU5,PT-1RWF,PT-1S6C,PT-1S7P,PT-1S19,PT-1S1W,PT-1S6F,PT-1RZY,PT-1S7O,PT-1S68,PT-1S2Y,PT-1RZG,PT-1SC7,PT-1S1D,PT-1S63,PT-1RZL,PT-1RV5,PT-1RU4,PT-1RVI,PT-1RZE,PT-1RWM,PT-1TB3,PT-1RV6,PT-1S2Z,PT-1S1Q,PT-1S6N,PT-BPYB,PT-8TKO,PT-FFLF,PT-8U7V
0,gene1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,gene2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,gene3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,gene4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,gene5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
dele.shape

(23343, 3087)

In [6]:
dele = dele.rename(columns = {"Unnamed: 0": "gene"})

In [7]:
# i is index, not gene number
all0 = list()
for i in range(dele.shape[0]):
    if sum(dele.iloc[i, 1:]) == 0:
        all0.append(i)

In [8]:
bound = list()
i = 0; j = 1; n_0 = len(all0)
while (j < n_0):
    if all0[j] - all0[i] >= 50:
        bound.append(all0[j])
        i = j
    j += 1

In [9]:
len(bound)

460

460 gene blocks for each sample.

In [10]:
sum(dele.iloc[200, 1:])

21

In [11]:
bound = [0] + bound[:-1] + [dele.shape[0]]

In [12]:
len(bound)

461

In [13]:
bound[:10], bound[-10:]

([0, 54, 104, 154, 208, 258, 308, 358, 408, 458],
 [22886, 22936, 22986, 23036, 23086, 23136, 23186, 23236, 23286, 23343])

In [14]:
tmp = dele.iloc[:, :5]

In [15]:
tmp.head()

,gene,PT-L1HP,PT-8TK1,PT-8UY5,PT-BQF5
0,gene1,0,0,0,0
1,gene2,0,0,0,0
2,gene3,0,0,0,0
3,gene4,0,0,0,0
4,gene5,0,0,0,0


In [16]:
tmp_df = pd.DataFrame(columns = tmp.columns[1:], index = [i for i in range(len(bound)-1)])

In [17]:
tmp_df.head()

,PT-L1HP,PT-8TK1,PT-8UY5,PT-BQF5
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [14]:
print (datetime.now().isoformat())
fill = list()
for i in range(len(bound)-1):
    fill_tmp = list()
    if bound[i]+1 > dele.shape[0]:
        print (i, bound[i]+1, dele.shape[1], "break")
        break
    for j in range(dele.shape[1]-1):
        fill_tmp.append(dele.iloc[bound[i]:bound[i+1], j+1].tolist())
    fill.append(fill_tmp)
print (datetime.now().isoformat())

2019-07-19T17:13:01.275901
2019-07-19T17:15:45.386719


In [15]:
dele_block = pd.DataFrame(fill)

In [16]:
dele_block.shape

(460, 3086)

In [21]:
# sum([len(i) for i in tmp_df["PT-L1HP"]])

In [34]:
# dele_block.to_csv(f"{cwd}/deletion_geneblock.gz", compression = "gzip", sep = "\t", header = False, index = False)

In [19]:
dele.shape

(23343, 3087)

In [20]:
dele_block.iloc[:10, :50].to_csv(f"{cwd}/minimal_working_example_block1.gz", compression = "gzip", index = False, header = False)